<a href="https://colab.research.google.com/github/Kirutkar/PathForge/blob/main/XG_Boost_Resume%20_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy==1.24.4 scikit-learn==1.3.2
import numpy as np
import sklearn
print(np.__version__)
print(sklearn.__version__)


1.24.4
1.3.2


In [ ]:
import pandas as pd

In [ ]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.84.0
    Uninstalling openai-1.84.0:
      Successfully uninstalled openai-1.84.0


In [ ]:
from google.colab import files
uploaded=files.upload()

Saving balanced_resume_dataset.csv to balanced_resume_dataset (1).csv


In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import resample

# 📥 Load your resume dataset
df = pd.read_csv("balanced_resume_dataset.csv")  # use your file here

# 📝 Combine 'summary' + 'skills' into one text column
df['combined_text'] = df['summary'].fillna('') + " " + df['skills'].fillna('')

# 🔑 Weighted keywords across multiple domains
weighted_keywords = {
    # 🔹 Advanced AI / Technical
    'llm': 3.5, 'langchain': 3.5, 'rag': 3.5, 'rag pipeline': 3.5,
    'vector db': 3.5, 'weaviate': 3, 'chromadb': 3, 'pinecone': 3,
    'agent': 3, 'langchain agents': 3.5, 'autonomous agent': 3,
    'fine-tuning': 3, 'embedding': 3, 'semantic search': 3,
    'transformers': 3, 'huggingface': 3, 'openai': 3,
    'streamlit': 2.5, 'flask': 2.5, 'gradio': 2.5,
    'pytorch': 2.5, 'tensorflow': 2.5,
    'sql': 2, 'power bi': 2, 'pandas': 2, 'numpy': 2,
    'data analysis': 2,

    # 🔸 Business / Management
    'project management': 3.5, 'agile': 3, 'stakeholder': 2.5,
    'scrum': 3, 'planning': 2, 'budgeting': 2,
    'strategic partnerships': 3, 'gtm': 2.5, 'account planning': 2.5,
    'market share': 2.5, 'revenue growth': 3, 'client relationships': 2.5,

    # 🟢 Sales / CRM
    'crm': 3, 'channel sales': 3, 'business development': 3,
    'partner engagement': 3, 'sales forecasting': 2.5,
    'campaign': 2.5, 'salesforce': 2.5, 'leads': 2, 'market research': 2.5,
    'negotiation': 2, 'presentation': 2,

    # 🟠 Education / Teaching
    'curriculum': 3, 'lesson planning': 2.5, 'teaching': 3,
    'student engagement': 2.5, 'learning outcomes': 2.5, 'training': 2.5,
    'academic institutions': 2.5, 'ministry of education': 3,
    'educational partnerships': 2.5, 'demo': 2, 'workshop': 2,

    # 🔵 HR / Support
    'recruitment': 3, 'employee engagement': 2.5, 'onboarding': 2.5,
    'conflict resolution': 2, 'policy': 2, 'human resources': 3,

    # 🔴 Security / Law Enforcement
    'security guard': 3, 'loss prevention': 3, 'cctv': 2.5, 'access control': 2.5,
    'conflict de-escalation': 2, 'law enforcement': 2.5, 'threat assessment': 2,
    'certified protection': 3, 'crisis intervention': 2, 'surveillance': 2
}

# 🎯 Scoring function
def weighted_score(text):
    text = text.lower()
    return sum(weight for kw, weight in weighted_keywords.items() if kw in text)

# 🔍 Apply scoring + normalize
df['resume_score'] = df['combined_text'].apply(weighted_score)
df['normalized_score'] = df['resume_score'] / np.log(df['combined_text'].apply(lambda x: len(x.split()) + 1))


# 📊 Dynamic thresholds
q_low = df['normalized_score'].quantile(0.25)
q_high = df['normalized_score'].quantile(0.75)

def map_strength(score):
    if score >= q_high:
        return 'Strong'
    elif score >= q_low:
        return 'Average'
    else:
        return 'Weak'

df['resume_strength'] = df['normalized_score'].apply(map_strength)

# ✅ Check distribution
print("🔍 Resume Strength Distribution (Before Balancing):")
print(df['resume_strength'].value_counts())

# ⚖️ Balance classes: 1000 each
df_weak = df[df['resume_strength'] == 'Weak']
df_avg = df[df['resume_strength'] == 'Average']
df_strong = df[df['resume_strength'] == 'Strong']

df_weak_down = resample(df_weak, replace=False, n_samples=1000, random_state=42)
df_avg_up = resample(df_avg, replace=True, n_samples=1000, random_state=42)
df_strong_up = resample(df_strong, replace=True, n_samples=1000, random_state=42)

df_balanced = pd.concat([df_weak_down, df_avg_up, df_strong_up])

# ✅ Save final output
df_balanced.to_csv("resume_strength_balanced.csv", index=False)
print("\n✅ Final balanced file saved as 'resume_strength_balanced.csv'")
print(df_balanced['resume_strength'].value_counts())


🔍 Resume Strength Distribution (Before Balancing):
resume_strength
Average    1969
Strong     1031
Weak       1000
Name: count, dtype: int64

✅ Final balanced file saved as 'resume_strength_balanced.csv'
resume_strength
Weak       1000
Average    1000
Strong     1000
Name: count, dtype: int64


In [ ]:
from google.colab import files
uploaded=files.upload()

Saving Synthetic_Resumes_Balanced.csv to Synthetic_Resumes_Balanced.csv


In [ ]:
original_df = pd.read_csv("resume_strength_balanced.csv")
synthetic_df = pd.read_csv("Synthetic_Resumes_Balanced.csv")


In [ ]:
combined_df = pd.concat([original_df, synthetic_df], ignore_index=True)


In [ ]:
combined_df.to_csv("combined_resume_dataset.csv", index=False)
print("✅ Combined dataset saved as 'combined_resume_dataset.csv'")
files.download("combined_resume_dataset.csv")

✅ Combined dataset saved as 'combined_resume_dataset.csv'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:

# ✅ STEP 1: Fill missing 'skills' and 'summary' (required before mapping)
combined_df['skills'] = combined_df['skills'].fillna('')
combined_df['summary'] = combined_df['summary'].fillna('')

# ✅ STEP 2: Define grouped_role mapping logic
def map_job_role(row):
    text = str(row['grouped_role']) + " " + str(row['skills']) + " " + str(row['summary'])
    text = text.lower()

    if any(word in text for word in ['developer', 'engineer', 'programmer']):
        return "Software Engineer"
    elif any(word in text for word in ['data', 'analyst', 'analytics']):
        return "Data Analyst"
    elif any(word in text for word in ['machine learning', 'ml', 'ai']):
        return "AI/ML Engineer"
    elif any(word in text for word in ['project manager', 'scrum', 'product']):
        return "Project/Product Manager"
    elif any(word in text for word in ['teacher', 'educator', 'instructor', 'mentor', 'trainer', 'edtech', 'curriculum', 'learning', 'educational']):
        return "Educator"
    elif any(word in text for word in ['hr', 'recruiter', 'talent']):
        return "HR/Recruiter"
    elif any(word in text for word in ['sales', 'crm', 'business development', 'marketing', 'funnel']):
        return "Sales/Marketing"
    elif any(word in text for word in ['support', 'helpdesk', 'customer success']):
        return "Support Staff"
    elif any(word in text for word in ['cloud', 'aws', 'azure', 'gcp']):
        return "Cloud Engineer"
    elif any(word in text for word in ['admin', 'coordinator', 'assistant']):
        return "Admin/Coordinator"
    elif 'intern' in text:
        return "Intern"
    else:
        return "Other"

# ✅ STEP 3: Apply the mapping only where grouped_role is missing
combined_df['grouped_role'] = combined_df.apply(
    lambda row: map_job_role(row) if pd.isna(row['grouped_role']) or row['grouped_role'] == '' else row['grouped_role'],
    axis=1
)

# ✅ STEP 4: Fill remaining essential columns
combined_df['user_id'] = combined_df['user_id'].fillna(method='ffill').fillna(method='bfill')
combined_df['experience_level'] = combined_df['experience_level'].fillna("Mid-Level")

# ✅ Final check
print("✅ Nulls After Mapping:")
print(combined_df.isnull().sum())

✅ Nulls After Mapping:
user_id              0
experience_level     0
skills               0
summary              0
grouped_role         0
combined_text       30
resume_score        30
normalized_score    30
resume_strength      0
dtype: int64


<ipython-input-11-3462833115>:42: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df['user_id'] = combined_df['user_id'].fillna(method='ffill').fillna(method='bfill')


In [ ]:
# STEP 1: Fill missing 'skills' with empty string (if any)
combined_df['skills'] = combined_df['skills'].fillna('')

# STEP 2: Combine summary and skills
combined_df['combined_text'] = combined_df['summary'].fillna('') + " " + combined_df['skills']

# STEP 3: Recalculate resume_score using your scoring function
def weighted_score(text):
    text = text.lower()
    return sum(weight for kw, weight in weighted_keywords.items() if kw in text)

combined_df['resume_score'] = combined_df['combined_text'].apply(weighted_score)

# STEP 4: Normalize by length
combined_df['normalized_score'] = combined_df['combined_text'].apply(lambda x: weighted_score(x) / np.log(len(x.split()) + 1))

# STEP 1: Fill missing 'skills' with empty string (if any)
combined_df['skills'] = combined_df['skills'].fillna('')

# STEP 2: Combine summary and skills
combined_df['combined_text'] = combined_df['summary'].fillna('') + " " + combined_df['skills']

# STEP 3: Recalculate resume_score using your scoring function
def weighted_score(text):
    text = text.lower()
    return sum(weight for kw, weight in weighted_keywords.items() if kw in text)

combined_df['resume_score'] = combined_df['combined_text'].apply(weighted_score)

# STEP 4: Normalize by length
combined_df['normalized_score'] = combined_df['combined_text'].apply(lambda x: weighted_score(x) / np.log(len(x.split()) + 1))

# 📊 Dynamic thresholds
q_low = df['normalized_score'].quantile(0.25)
q_high = df['normalized_score'].quantile(0.75)


# STEP 5: Reapply strength mapping
def map_strength(score):
    if score >= q_high:
        return 'Strong'
    elif score >= q_low:
        return 'Average'
    else:
        return 'Weak'

combined_df['resume_strength'] = combined_df['normalized_score'].apply(map_strength)

# ✅ Final check
print(combined_df.isnull().sum())



user_id             0
experience_level    0
skills              0
summary             0
grouped_role        0
combined_text       0
resume_score        0
normalized_score    0
resume_strength     0
dtype: int64


In [ ]:
combined_df.to_csv("final_resume_dataset.csv", index=False)

In [ ]:
files.download("final_resume_dataset.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving final_resume_dataset.csv to final_resume_dataset (1).csv


In [ ]:
import pandas as pd

# Load your cleaned dataset where 'resume_strength' column is already present
df = pd.read_csv("final_resume_dataset.csv")  # This should have 'summary', 'skills', and 'resume_strength'

# Combine 'summary' + 'skills' into one text field
df['combined_text'] = df['summary'].fillna('') + " " + df['skills'].fillna('')


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert text to numerical features
vectorizer = TfidfVectorizer(max_features=3000)
X = vectorizer.fit_transform(df['combined_text'])

# Target labels
y = df['resume_strength']


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier  # You can also use XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = GradientBoostingClassifier()
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("🎯 Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Save everything for real-time prediction
joblib.dump(model, "xgb_resume_model.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")


🎯 Accuracy: 0.9224422442244224
              precision    recall  f1-score   support

     Average       0.89      0.86      0.88       196
      Strong       0.99      0.94      0.96       197
        Weak       0.89      0.96      0.93       213

    accuracy                           0.92       606
   macro avg       0.92      0.92      0.92       606
weighted avg       0.92      0.92      0.92       606



['tfidf_vectorizer.pkl']

In [ ]:
print("q_low:", df['normalized_score'].quantile(0.25))
print("q_high:", df['normalized_score'].quantile(0.75))


q_low: 0.5166355336146575
q_high: 2.831921823997124
